# &#128214; Lab 4: Data Flow Analysis (DFA)

# &#128221; Exercise 1: Simple Data Flow Analysis

## &#127919; Objective
In this exercise, we will delve into a fundamental aspect of program analysis known as Data Flow Analysis. 
This technique is important for understanding how data moves and transforms across a program. 
We will use a well-known analysis known as the Reaching Definition Analysis to illustrate the concepts of data flow analysis. 
This will provide a foundation for understanding how different definitions of a variable can reach various points in a program.

---

## &#128214; Background

Data Flow Analysis is a method used to gather valuable information about a program's execution. 
It examines the transformations and movement of data across the program, focusing on how the data values change and interact over the program's control flow graph.

Reaching Definition Analysis is a specific form of data flow analysis that aims to determine at each point in the program, the set of variable definitions that may have reached that point. 
It's a forward analysis that propagates definition information from the start to the end of the program.

Here are some key terms associated with this analysis:
- **Definition:** A statement where a variable is assigned a new value.
- **Reaching Definition:** A definition that can reach a particular point in the program without any re-definition of the variable.
- **Gen Set:** The set of definitions generated by a statement.
- **Kill Set:** The set of definitions killed (or invalidated) by a statement.

### &#128161; Gen and Kill Sets in Dataflow Analysis

In dataflow analysis, "gen" and "kill" sets are fundamental concepts used to model how information flows through a program.

- **Gen Set**: Represents the information *generated* at a particular program point. It contains facts that are true after the program statement is executed.

- **Kill Set**: Represents the information *invalidated* or *killed* at that program point. It contains facts that are no longer true after the statement is executed.

The gen and kill sets help in computing "in" and "out" sets for each program point, providing a basis for various optimizations and analyses.

### &#128161; In and Out Sets in Dataflow Analysis

In addition to gen and kill sets, "in" and "out" sets are crucial for understanding how data flows through a program.

- **In Set**: Represents the set of *facts* that hold true just before a particular program point is reached. It is computed based on the out sets of the program points that immediately precede it.

- **Out Set**: Represents the set of *facts* that hold true just after the program point is executed. It is computed as a function of the in set and the gen and kill sets at that program point. The formula for computing the out set is generally:  
  `Out = Gen U (In - Kill)`

&#10145; The computation of the `Out` set in this case is a function of the `In`, `Gen`, and `Kill` sets.
This function is called a **Flow Function**.

In the context of data flow analysis, a **flow function** is a mathematical abstraction that models the effect of a program statement or a basic block on the data flow information. 

The flow function `f` is generally defined as:

`f(In) = Gen U (In - Kill)`

The specific nature of these sets and the flow function itself will vary depending on the type of data flow analysis being conducted, such as reaching definitions, live variables, or available expressions analysis.

The in and out sets are calculated iteratively until they reach a fixed point, which enables various static analyses and optimizations.

### &#128161; Fact in Dataflow Analysis

In the context of dataflow analysis, a "fact" is a piece of information that holds true at a specific point in a program (e.g., a fact could be: *this variable is null at this program point*, or: *this variable equals exactly 10 at this program point*).
Facts can represent various properties such as the availability of an expression, reaching definitions, or the possible values a variable can take. 
They are the building blocks used in gen and kill sets to propagate information through the program, facilitating various types of static analyses.


## Reaching Definition Analysis

Reaching Definition Analysis is a fundamental data flow analysis that provides a basis for many optimization and analysis techniques in compiler design and program analysis. 
Its primary utility lies in determining what definitions of variables may be used at various points in a program, which can, in turn, help in various optimizations and analyses such as:

1. **Dead Code Elimination:** By identifying definitions that don't reach any use, we can identify and eliminate dead code.
2. **Constant Propagation:** By tracking the definitions that reach each point, we can propagate constants and simplify expressions.

---

### Example:

```python
x = 10          # Definition D1
y = 20          # Definition D2
x = 30          # Definition D3
z = x + y       # Use U1 of x and y
```

In this code snippet:

- Initially, `x` is defined as `10` (Definition D1) and `y` is defined as `20` (Definition D2).
- Then `x` is re-defined as `30` (Definition D3), overriding the previous definition D1.

Now, let's analyze the reaching definitions at different points in this program:

1. Before the statement `x = 30`:
   - The set of reaching definitions is `{D1, D2}`.
2. Before the statement `z = x + y`:
   - The set of reaching definitions is `{D2, D3}` as `D3` has overridden `D1`.

At the use `U1` of `x` and `y` in the expression `x + y`, the reaching definitions for `x` and `y` are `D3` and `D2` respectively. Definition `D1` for `x` is no longer reaching this point as it has been overridden by `D3`.

This example illustrates why the set of reaching definitions changes as we progress through the code, and how the latest definition of a variable at any point in the code is the one that reaches that point, provided no other definitions of the same variable occur in between.

---

## &#10145; Tasks

1. **Understanding Reaching Definition Analysis:**
   - Familiarize yourself with the concept of reaching definitions and how it is used in data flow analysis.

2. **Implementing Reaching Definition Analysis:**
   - Use the `ControlFlowGraph` class to support the computation of reaching definitions.
   - Implement a worklist algorithm to compute the in-sets and out-sets of definitions for each node in the CFG.
   - Verify your implementation with simple examples to see the reaching definitions at each point in the program.

### Import the necessary library

&#128161; *In the following cell, we will import the library needed for this exercise:*
- `ast`: a module of the python standard library to transform Python code in its AST representation
- `graphviz`: a library to create directed graphs

In [ ]:
import ast
import graphviz

Python code

&#128161; The following cell contains a string that represents the Python code that will be analyzed through this exercise

In [ ]:
code = """
x = 10          # Definition D1
y = 20          # Definition D2
x = 30          # Definition D3
z = x + y       # Use U1 of x and y
"""

### Utility Control Flow Graph class

&#128161; The following cell contains a utility class to build a Control Glow Graph. 
You have to use this class to build the control flow graph.

In [ ]:
class ControlFlowGraph:
    """
    A class representing a Control Flow Graph (CFG).

    Attributes:
    nodes: A list where each element is a statement.
    edges: A list of tuples representing edges between nodes, where each tuple contains a pair of nodes.

    Methods:
    add_node:
        Adds a new node with the given statement to the graph, returning the new node.
    add_edge:
        Adds an edge between the specified node indices to the graph.
    visualize:
        Prints a visualization of the graph to the console.
    to_dot:
        Returns a DOT-format string representing the graph.
    """

    def __init__(self):
        """
        Initializes a new instance of the ControlFlowGraph class, with empty nodes and edges.
        """
        self.nodes = []
        self.edges = []

    def add_node(self, statement):
        """
        Adds a new node with the given statement to the graph.

        Parameters:
        statement: The statement associated with the new node.

        Returns:
        The node
        """
        self.nodes.append(statement)
        return statement

    def add_edge(self, node1, node2):
        """
        Adds an edge between the specified nodes to the graph.

        Parameters:
        node1: The source node.
        node2: The target node.
        """
        if (node1, node2) not in self.edges:
            self.edges.append((node1, node2))

    def visualize(self):
        """
        Prints a visualization of the graph to the console.
        Each edge is printed as a line in the format 'source -> target'.
        """
        for edge in self.edges:
            source, target = edge
            print(f'{source} -> {target}')

    def to_dot(self):
        """
        Returns a DOT-format string representing the graph (for vizualization purposes).

        Returns:
        str: A string in DOT format.
        """
        dot_lines = ['digraph cfg {']
        stmt_to_index = {}
        for index, statement in enumerate(self.nodes):
            stmt_to_index[statement] = index
            dot_lines.append(f'    node{index} [label="{statement}"];')
        for edge in self.edges:
            source, target = edge
            dot_lines.append(f'    node{stmt_to_index[source]} -> node{stmt_to_index[target]};')
        dot_lines.append('}')
        return '\n'.join(dot_lines)

### Function to create the CFG

&#128161; In the following cell, you will use the `build_cfg` function of the previous lab.

In [ ]:
def build_cfg(node, cfg, parent=None):
    """
    Recursively builds a Control Flow Graph (CFG) from an Abstract Syntax Tree (AST) node, considering various types of statements including assignment, expression, conditional, loop, and module level statements.

    Parameters:
    node: The current node in the AST.
    cfg: An instance of ControlFlowGraph to which nodes and edges will be added.
    parent: A list of parent nodes from which edges to the current node will be drawn. 
    
    Returns:
    list: A list of current nodes which will act as parents for the next level of recursion.
    """


### Build the CFG

&#128161; In the following cell, you will build the CFG as in the previous lab

### Implement the Reaching Definition Analysis

&#128161; In the following cell, you will implement the *reaching definitions analysis* using a class called `ReachingDefinitionsAnalysis`. 
The class should have fields representing the `in` and `out` sets for each of the nodes of the CFG so that we can query those after the analysis.

&#10145; The main algorithm should be implemented using a *worklist* algorithm.

For simplicity we only consider definition statements as `ast.Assign`

In [ ]:
class ReachingDefinitionsAnalysis:
    """
    A class to perform reaching definitions analysis on a Control Flow Graph (CFG).

    Attributes:
        cfg: The Control Flow Graph on which to perform the analysis.
        in_sets: A dictionary to store the 'in' sets for each node.
        out_sets: A dictionary to store the 'out' sets for each node.

    Methods:
        analyze: Performs the reaching definitions analysis.
    """

    def __init__(self, cfg):
        """
        Initialize the ReachingDefinitions class.

        Parameters:
            cfg: The Control Flow Graph on which to perform the analysis.
        """
        self.cfg = cfg
        self.in_sets = {node: set() for node in cfg.nodes}
        self.out_sets = {node: set() for node in cfg.nodes}

    def analyze(self):
        """
        Perform the reaching definitions analysis using a worklist algorithm.
        
        Updates the in_sets and out_sets attributes with the analysis results.
        """
        first_id = 1
        worklist = [self.cfg.nodes[first_id]]
        visited = set()

        while worklist:
            node = worklist.pop(0)
            visited.add(node)


#### Test your code

&#128161; In the following cell, you will instantiate a `ReachingDefinitionsAnalysis` object with the CFG as a parameter and trigger the reaching analysis that should populate the `in` and `out` sets for each node of the CFG.

#### Check the analysis results

&#128161; In the following cell, you will iterate over the `in` and `out` sets of the ReachingDefinitionsAnalysis object and, for each node of the CFG, print their in and out sets.

### &#10067; Questions

#### Were you able to get it right?

#### What is the output supposed to be?

#### Does it match the output of your program?

### Abstraction

&#129300; One might question the necessity of utilizing classes and fields instead of opting for a simple function-based approach.

The reason behind this architectural choice is generalization. The core structure and algorithm we employed can be adapted for diverse data flow analyses. Our specific problem—reaching definitions analysis—does have unique characteristics, such as the functions used to generate the `gen` and `kill` sets. But the rest is general, the flow function is always the same:

`f(In) = Gen U (In - Kill)`

By adopting a class-based structure, we can establish a uniform framework for various analyses. This allows for the flexible instantiation of components specific to each type of analysis.

&#10145; In essence, while the foundational problem remains constant, the instantiation can differ, offering greater adaptability.

In the following cell, you will provide a class to generalize any data flow analysis, only keep what is general to any data flow problem (i.e., `in` and `out` sets, the main algorithm, etc.). The flow function itself also is general, just the computation of `gen` and `kill` are different.

In [ ]:
from abc import ABC, abstractmethod

class DataFlowAnalysis(ABC):
    """
    Abstract Base Class for data flow analysis on a Control Flow Graph (CFG).

    Attributes:
        cfg: Control Flow Graph on which to perform the analysis.
        in_sets: Dictionary to store 'in' sets for each node.
        out_sets: Dictionary to store 'out' sets for each node.
    """

    def __init__(self, cfg):
        """
        Initialize the DataFlowAnalysis class.

        Parameters:
            cfg: Control Flow Graph on which to perform the analysis.
        """

        self.cfg = cfg
        self.in_sets = {node: set() for node in cfg.nodes}
        self.out_sets = {node: set() for node in cfg.nodes}

    def analyze(self):
        """
        Core algorithm for data flow analysis, common to all types of analyses.
        
        Updates the in_sets and out_sets attributes based on the implemented get_gen and get_kill methods.
        """

        first_id = 1
        worklist = [self.cfg.nodes[first_id]]
        visited = set()

        while worklist:
            node = worklist.pop(0)
            visited.add(node)

Good, now, in the following cell, you will re-implement the `ReachingDefinitionsAnalysis` class, but this time it should extend the `DataFlowAnalysis` class and implement the methods it should.

In [ ]:
class ReachingDefinitionsAnalysis(DataFlowAnalysis, ABC):
    def __init__(self, cfg):
        super().__init__(cfg)

#### Test your code

&#128161; In the following cell, you will instantiate a `ReachingDefinitionsAnalysis` object and trigger the analysis.

#### Check the analysis results

&#128161; In the following cell, you will iterate over the `in` and `out` sets of the ReachingDefinitionsAnalysis object and, for each node of the CFG, print their in and out sets.

&#10067; Were you able to get the same results as with the solution without abstraction?

You should have.

### Let us try with a more complicated example with branches

Python code

&#128161; The new code on which you will apply your analyses

In [ ]:
code = """
x = 10
y = 20
x = 30
if x < 10:
    y = 4
z = x + y
"""

Build the CFG and reason on the CFG, what should be the reaching definition for each statement?

#### Apply the reaching definition analysis on the new piece of code

### &#10067; Questions

#### Does it match what it is supposed to be?

#### According to you, why?

#### What would you improve?

#### According to you, why?

#### What about loops, what can be the impact of loops?

To perform real data flow analysis on real-world piece of code, we need techniques that work for any kind of statement, we need a *Data Flow Framework*

If you want to know more about that, feel free to consult online resources.

A good place to start is: https://cs.au.dk/~amoeller/spa/